### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению, 
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

## Данные:
### Директория где находятся исходные данные 
/home/al/PycharmProjects/examples_problem_solving_kaggl/mashin_learning_and_kaggle/jupiter_bilder/source_data/<br>

### сами данные: 
train.csv - данные по энергопотреблению <br>

building_metadata.csv - данные по зданиям<br>

weather_train.csv - данные о погоде <br>


## Соревнование:
https://www.kaggle.com/c/ashrae-energy-prediction/

### Загрузка библиотек

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8

### Загрузка данных
Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [5]:
ish_data = pd.read_csv("/home/al/PycharmProjects/examples_problem_solving_kaggl/mashin_learning_and_kaggle/jupiter_bilder/train.csv")
energy_0 = ish_data.loc[ish_data['building_id'] == 0]
energy_0 = energy_0[energy_0["meter_reading"] > 0]
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())

         building_id  meter           timestamp  meter_reading  hour
1617286            0      0 2016-01-30 08:00:00        43.6839     8
1665732            0      0 2016-01-31 05:00:00        37.5408     5
1692691            0      0 2016-01-31 17:00:00        52.5571    17
5205692            0      0 2016-04-08 14:00:00        59.3827    14
6509935            0      0 2016-05-01 19:00:00       448.0000    19


### Разделение данных на обучение и проверку
Выделим 20% всех данных на проверку, остальные оставим на обучение

In [6]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

          building_id  meter           timestamp  meter_reading  hour
16023341            0      0 2016-10-18 04:00:00        283.945     4
11217636            0      0 2016-07-24 11:00:00        297.596    11
17703727            0      0 2016-11-17 02:00:00        186.339     2
8415486             0      0 2016-06-04 16:00:00        244.357    16
15045830            0      0 2016-09-30 21:00:00        246.404    21


### Создадим модели
Среднее и медианное значение потребление энергии по часам

In [7]:
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

         Среднее   Медиана
hour                      
0     239.939537  240.6025
1     240.189594  240.9440
2     239.802408  242.9915
3     243.111480  245.3805
4     239.096864  244.3570
5     239.756871  245.0390
6     238.952998  246.4040
7     241.493654  247.0870
8     241.759961  239.5790
9     235.845879  234.8005
10    236.595329  239.5790
11    236.123651  241.6260
12    231.850966  240.9440
13    239.288355  244.6980
14    237.070923  242.3090
15    235.400445  241.6260
16    233.763553  240.9440
17    233.413140  239.5790
18    237.557504  238.8960
19    238.438632  238.2140
20    238.716299  236.1660
21    238.430894  238.8960
22    238.281446  238.8960
23    238.687716  239.5790


### Функция проверки модели
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [8]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())

          building_id  meter           timestamp  meter_reading  hour  \
9930979             0      0 2016-07-01 14:00:00        255.960    14   
18747623            0      0 2016-12-06 01:00:00        182.926     1   
9670584             0      0 2016-06-26 23:00:00        237.531    23   
17657639            0      0 2016-11-16 06:00:00        176.101     6   
14092938            0      0 2016-09-14 00:00:00        245.722     0   

          meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
9930979               0.005830                0.002980             30.790518  
18747623              0.073468                0.075171             27.191360  
9670584               0.000023                0.000073             29.970142  
17657639              0.092248                0.111758             26.798432  
14092938              0.000562                0.000440             30.340952  


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [9]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.2539681441823892
Качество среднего: 0.25199623814522715
Качество нуля: 5.44883382023213
